In [1]:
import ee
import geemap
import ipywidgets as widgets
import datetime as dt

#ee.Initialize()
#ee.Authenticate()

In [2]:
Map = geemap.Map()

In [3]:
#Cloud mask
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

#Landsat image scale fator
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)

In [6]:
#Widget to select the dataset
ds_wgt = widgets.Dropdown(
    options=['Sentinel', 'Landsat', 'Modis'],
    value='Sentinel',
    description="Dataset:")
    #layout=widgets.Layout(width=widget_width, padding=padding),
    #style={"description_width": "initial"})

#Widget date
date1_wgt = widgets.DatePicker(
    description='Start Date:',
    value=dt.date(2023, 1, 1),
    disabled=False)

date2_wgt = widgets.DatePicker(
    description='End Date:',
    value=dt.date(2023, 1, 1),
    disabled=False)
    
date_box = widgets.HBox([date1_wgt, date2_wgt])

#Widget layer name
name = widgets.Text(
    value='',
    placeholder='Layer Name',
    description='Layer Name',
    disabled=False)
    
#Widget to execute function run_map()    
button = widgets.Button(
    description='Load Parameters', 
    button_style='primary',
    tooltip='Click me')
output = widgets.Output()
button_box = widgets.HBox([button, output])

In [7]:
def date1():    
    ss = date1_wgt.value
    date1 = "{}-{:02}-{:02}".format(ss.year, ss.month, ss.day)
    return(date1)
    
def date2():
    ss = date2_wgt.value
    date2 = "{}-{:02}-{:02}".format(ss.year, ss.month, ss.day)
    return(date2)

#Getting roi
def roi():
    roi = Map.draw_features[0]
    roi = ee.Feature(roi).geometry()
    return(roi)

def run_map(b):  
    #Loading datasets to map
    if ds_wgt.value == 'Sentinel':
        dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filterDate(date1(), date2()) \
                  .filterBounds(roi())\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
                  .map(maskS2clouds)\
                  .mean()           

        visualization = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2']
        }
            
        Map.addLayer(dataset, visualization, name.value)
        
    if ds_wgt.value == 'Landsat':
        dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
            .filterDate(date1(), date2())\
            .filterBounds(roi())
    
        dataset = dataset.map(applyScaleFactors)

        visualization = {
            'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
            'min': 0.0,
            'max': 0.3
            }
        
        Map.addLayer(dataset, visualization, name.value)
        
    if ds_wgt.value == 'Modis':
        dataset = ee.ImageCollection('MODIS/006/MOD09GQ') \
            .filter(ee.Filter.date(date1(), date2()))\
            .filterBounds(roi())
    
        visualization = {
        'min': -100.0,
        'max': 8000.0,
        'bands': ['sur_refl_b02', 'sur_refl_b02', 'sur_refl_b01'],}
        
        Map.addLayer(dataset, visualization, name.value)

button.on_click(run_map)

In [8]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
widgets.HBox([date_box, ds_wgt, name])

In [10]:
button_box